<a href="https://colab.research.google.com/github/ShannonH98/PyTorch_Practice_Projects/blob/main/Cat_vs_Dog_Classifier_02_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

currently a work in progress

# First Set up the environment and get the data

In [ ]:
import torch
from torch import nn

# Note: this notebook requires torch >= 1.10.0
torch.__version__

In [ ]:
!pip install kaggle

In [ ]:
!pip install opendatasets

In [ ]:
#device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
import opendatasets as od
import pandas

od.download("https://www.kaggle.com/datasets/shaunthesheep/microsoft-catsvsdogs-dataset")

In [ ]:
#walk through the downloaded directory
import os

def walk_through_dir(dir_path):
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [ ]:
image_path = "microsoft-catsvsdogs-dataset"
walk_through_dir(image_path)

# Visualize the data randomly to ensure it was dowloaded successfully.

In [ ]:
import random
from PIL import Image
import glob
from pathlib import Path

# Set seed
#random.seed(42)

# 1. Get all image paths (* means "any combination")
image_path_list= glob.glob(f"{image_path}/*/*/*.jpg")

# 2. Get random image path
random_image_path = random.choice(image_path_list)

# 3. Get image class from path name (the image class is the name of the directory where the image is stored)
image_class = Path(random_image_path).parent.stem

# 4. Open image
img = Image.open(random_image_path)

# 5. Print metadata
print(f"Random image path: {random_image_path}")
print(f"Image class: {image_class}")
print(f"Image height: {img.height}")
print(f"Image width: {img.width}")
img

In [ ]:
#visualize with matplot lib

#Visualizing the Data
import matplotlib.pyplot as plt


plt.imshow(img)

# Optional: Add a title
plt.title(f"Image class: {image_class}")

# Optional: Remove axes ticks for a cleaner image display
plt.axis('off')

# 3. Show the plot
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

img_as_array = np.asarray(img)

plt.figure(figsize=(10, 7))
plt.imshow(img_as_array)
plt.axis(False)
plt.title(f"image class: {image_class}, image shape{img_as_array.shape}") #this would be in the format height, width, color
plt.show()

In [ ]:
img_as_array

# Now we need to split the data into train, test and validate batches.


In [ ]:
folder_path = "/content/microsoft-catsvsdogs-dataset/PetImages/Dog"

file_count = 0
for root, _, files in os.walk(folder_path):
    file_count += len(files)

print(f"Total number of files (including subdirectories) in '{folder_path}': {file_count}")

In [ ]:
base_dir = "dataset_split"
categories = ["Cat", "Dog"]
splits = ["train", "val", "test"]

In [ ]:
for split in splits:
    for category in categories:
        folder_path = os.path.join(base_dir, split, category)
        os.makedirs(folder_path, exist_ok=True)

In [ ]:
#This makes different folders where I will put the test, train and validation folders
counts = {split: [] for split in splits}

for split in splits:
    for category in categories:
        folder = os.path.join(base_dir, split, category)
        counts[split].append(len(os.listdir(folder)))

print("Counts (train/val/test) for Cat and Dog:", counts)

In [ ]:
import shutil

split_ratio = {"train": 0.7, "val": 0.15, "test": 0.15}
#70% to teain, 15 to validation and 15 to test

for category in categories:
    src_folder = f"/content/microsoft-catsvsdogs-dataset/PetImages/{category}"
    images = os.listdir(src_folder)
    random.shuffle(images)  # shuffle for randomness
    total = len(images)

    train_end = int(split_ratio["train"] * total)
    val_end = train_end + int(split_ratio["val"] * total)

    splits_images = {
        "train": images[:train_end],
        "val": images[train_end:val_end],
        "test": images[val_end:]
    }

    # Copy images to the new folders
    for split_name, split_images in splits_images.items():
        for img in split_images:
            shutil.copy(os.path.join(src_folder, img),
                        os.path.join(base_dir, split_name, category, img))

In [ ]:
#Visualize the total in each group
print(counts)

In [ ]:
import numpy as np

x = np.arange(len(categories))  # Cat, Dog
width = 0.20  # width of bars

fig, ax = plt.subplots()
ax.bar(x - width, counts["train"], width, label='Train')
ax.bar(x, counts["val"], width, label='Validation')
ax.bar(x + width, counts["test"], width, label='Test')

ax.set_ylabel('Number of Images')
ax.set_title('Dataset split counts')
ax.set_xticks(x) # Set the tick locations explicitly
ax.set_xticklabels(categories)
ax.legend()
plt.show()

# Now, we need create data transform

In [ ]:
from PIL import Image
import os

def verify_and_clean_dataset(root_folder):
    """
    Walk through all image files and remove any that can't be opened.
    """
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    img.verify()  # verify that it is, in fact, an image
            except (IOError, SyntaxError) as e:
                print(f"Removing bad file: {file_path} ({e})")
                os.remove(file_path)

# Run it once on your dataset folders:
verify_and_clean_dataset("dataset_split/train")
verify_and_clean_dataset("dataset_split/val")
verify_and_clean_dataset("dataset_split/test")


In [ ]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import warnings
from PIL import Image

# 1. Clean dataset
verify_and_clean_dataset("dataset_split/train")
verify_and_clean_dataset("dataset_split/val")
verify_and_clean_dataset("dataset_split/test")

# 2. Define transforms
data_transform = transforms.Compose([
    transforms.Resize((128, 128)),   #resize all images to 128, 128
    transforms.ToTensor(), #convert image to tensor data
    transforms.Normalize([0.5], [0.5],) # Normalize pixel values
])

# 3. Create datasets
train_dataset = datasets.ImageFolder("dataset_split/train", transform=data_transform)
val_dataset = datasets.ImageFolder("dataset_split/val", transform=data_transform)

# 4. Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)


#warnings.filterwarnings("ignore", category=UserWarning, module="PIL.TiffImagePlugin")


In [ ]:
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

data_transform = transforms.Compose([
    transforms.Resize((128, 128)),   #resize all images to 128, 128
    transforms.ToTensor(), #convert image to tensor data
    transforms.Normalize([0.5], [0.5],) # Normalize pixel values
])

def safe_loader(path):
    from PIL import Image
    try:
        with open(path, 'rb') as f:
            img = Image.open(f)
            return img.convert('RGB')
    except:
        print(f"Skipping bad image: {path}")
        return None


In [ ]:
#now to load the dataset

from torchvision.datasets import ImageFolder

def safe_loader(path):
    try:
        with open(path, 'rb') as f:
            img = Image.open(f)
            return img.convert('RGB')  # force RGB
    except Exception as e:
        print(f"Skipping bad image: {path} (error: {e})")
        return None

train_dataset = ImageFolder(root="dataset_split/train", transform=data_transform, loader=safe_loader)
val_dataset   = ImageFolder(root="dataset_split/val", transform=data_transform, loader=safe_loader)
test_dataset  = ImageFolder(root="dataset_split/test", transform=data_transform, loader=safe_loader)


In [ ]:
train_dataset, val_dataset, test_dataset

In [ ]:
from torch.utils.data import DataLoader

batch_size = 32

def collate_fn(batch):
    batch = [b for b in batch if b[0] is not None]  # keep only valid (img, label) pairs
    if len(batch) == 0:   # if a whole batch is empty, skip it
        return torch.Tensor()
    return torch.utils.data.dataloader.default_collate(batch)

# dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)


# Define simple CNN for this classification

In [ ]:
import torch.nn.functional as F

class CatDogCNN(nn.Module):
    def __init__(self):
        super(CatDogCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)  # First convolutional layer
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)               # Downsampling
        # Adjust the input size of the linear layer based on the output size of the last pooling layer
        self.fc1 = nn.Linear(32 * 32 * 32, 1)         # Fully connected layer for binary output

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Conv1 -> ReLU -> Pool
        x = self.pool(F.relu(self.conv2(x)))  # Conv2 -> ReLU -> Pool
        x = x.view(x.size(0), -1)          # Flatten before FC, using x.size(0) for batch size
        return torch.sigmoid(self.fc1(x))     # Sigmoid for binary output

model = CatDogCNN()
model.to(device)

In [ ]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
from PIL import Image
import os

def check_images(folder):
    for root, _, files in os.walk(folder):
        for f in files:
            path = os.path.join(root, f)
            try:
                img = Image.open(path)
                img.verify()  # verify that it's an image
            except (IOError, SyntaxError) as e:
                print("Corrupt image:", path)

check_images("dataset_split")

An error i ran into in this case is corrupted images. From research it is very common for cat vs dog databases to have corrpted images. So i need to check for it in the imported database.

In [ ]:
from torchvision.datasets import ImageFolder

train_dataset = ImageFolder("dataset_split/train", transform=data_transform,
                            is_valid_file=lambda x: True)  # optional


In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for inputs, labels in train_loader:
        labels = labels.float().unsqueeze(1)
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = (correct / total) * 100
    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Loss: {running_loss/len(train_loader):.4f}, "
          f"Acc: {train_acc:.2f}%")